<a href="https://colab.research.google.com/github/Mamun-Developer/Thesis-Project/blob/master/Thesis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The Notebook Requres 30 min to train and test.
#### Accuracy can be improved by reducing features
#### Current accuracy is not that much good which is 67-69 %

In [1]:
!pip install numpy
!pip install pandas
!pip install nltk
!pip install sklearn

In [2]:
%%time
import numpy as np
import pandas as pd
import re
import time
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords

Wall time: 1.14 s


In [3]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 1. Reading tweets from file

In [4]:
%%time
#tweets = pd.read_csv("updated data for tfidf.csv")
#Uncommnet next line when working
tweets = pd.read_csv("cleanTweets.csv")
#tweets = tweets[0:1000]

Wall time: 40.9 ms


In [5]:
tweets

,sentiment,tweet
0,neutral,I am saying @Man BBQ prty is going ton8. Gd......
1,positive,The striped bats are hanging on their feet for...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,positive,@AmericanAir thank you we got on a different f...
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...
14637,neutral,@AmericanAir Please bring American Airlines to...
14638,negative,"@AmericanAir you have my money, you change my ..."


### 2. Seperating Features and labels

In [6]:
features = tweets["tweet"]

labels = tweets["sentiment"]

### 3. Preprocessing features:
    a. Remove hyperlinks
    b. Remove Reply tags
    c. Remove special characters and extra spaces
    c. Remove all single character
    d. Convert to lowercase

In [7]:
%%time
processed_features = []

for sentence in range(0, len(features)):
    
    
    # Remove reply tag
    processed_feature = re.sub("(@[A-Za-z0-9_]+)", "", str(features[sentence]))
    
    #Remove Hyperlink
    processed_feature = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|' \
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', processed_feature)
    
    # Remove ' from sentence
    processed_feature = re.sub(r"'", '', processed_feature)
    
    # Remove all special characters including multiple spaces replaced with single space
    processed_feature = re.sub(r'\W', ' ', processed_feature)
    
    # remove all single characters (will miss first and last single character if no space in first and last)
    processed_feature= re.sub(r'\s+[a-zA-Z1-9]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'^[a-zA-Z1-9]\s+', '', processed_feature) 
    
    # Remove single characters from the end
    processed_feature = re.sub(r'\s+[a-zA-Z1-9]$', '', processed_feature) 
    
    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature)

    # Removing spaces from first and last
    processed_feature = processed_feature.strip()
    
    # Converting to Lowercase
    processed_feature = processed_feature.lower()
    
    processed_features.append(processed_feature)

Wall time: 695 ms


In [8]:
%%time
# After basic preprocessing
pd.DataFrame(processed_features).head(10)

Wall time: 2 ms


,0
0,am saying bbq prty is going ton8 gd n8 man
1,the striped bats are hanging on their feet for...
2,didnt today must mean need to take another trip
3,its really aggressive to blast obnoxious enter...
4,and its really big bad thing about it
5,seriously would pay 30 flight for seats that d...
6,yes nearly every time fly vx this ear worm won...
7,really missed prime opportunity for men withou...
8,well didnt but now do
9,it was amazing and arrived an hour early youre...


### 4.  Remove StopWords and Numbers

In [9]:
%%time
lemmaTokens = []
rawTokens = []
lemmatizer = WordNetLemmatizer()

# Tokenizing
for sentence in processed_features:
    tempTokens = sentence.split()
    rawTokens.append(tempTokens)
    lemmatizedTokens = []
    
    # proceed lemmatization
    for singleToken, tag in pos_tag(tempTokens):
        # Preparing for morphological analysis
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
            
        token = lemmatizer.lemmatize(singleToken, pos)
        
        if token not in stopwords.words("english"):
            if token.isdigit() == False:
                lemmatizedTokens.append(token)
        
    # append the lemmatized tokens of sentence in the list
    lemmaTokens.append(lemmatizedTokens)

Wall time: 1min 14s


In [10]:
%%time
# After Modification
pd.DataFrame(lemmaTokens[0:5])

Wall time: 1.99 ms


,0,1,2,3,4,5,6,7,8,9
0,say,bbq,prty,go,ton8,gd,n8,man,None,None
1,striped,bat,hang,foot,best,None,None,None,None,None
2,didnt,today,must,mean,need,take,another,trip,None,None
3,really,aggressive,blast,obnoxious,entertainment,guest,face,amp,little,recourse
4,really,big,bad,thing,None,None,None,None,None,None


In [11]:
# Total Document including empty document
len(lemmaTokens)

14640

In [12]:
# Removing Empty Document
count = 0
for tok in lemmaTokens:
    if len(tok)==0:
        lemmaTokens.pop(count)
        rawTokens.pop(count)
        labels.pop(count)
        
    count+=1

In [14]:
# Total Document after removing empty document
len(lemmaTokens)

14605

In [15]:
# Total Number of Labels
len(labels)

14605

In [16]:
%%time
# Before Modification
pd.DataFrame(rawTokens)

Wall time: 74.8 ms


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,am,saying,bbq,prty,is,going,ton8,gd,n8,man,...,None,None,None,None,None,None,None,None,None,None
1,the,striped,bats,are,hanging,on,their,feet,for,best,...,None,None,None,None,None,None,None,None,None,None
2,didnt,today,must,mean,need,to,take,another,trip,None,...,None,None,None,None,None,None,None,None,None,None
3,its,really,aggressive,to,blast,obnoxious,entertainment,in,your,guests,...,None,None,None,None,None,None,None,None,None,None
4,and,its,really,big,bad,thing,about,it,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14600,thank,you,we,got,on,different,flight,to,chicago,None,...,None,None,None,None,None,None,None,None,None,None
14601,leaving,over,20,minutes,late,flight,no,warnings,or,communication,...,svc,None,None,None,None,None,None,None,None,None
14602,please,bring,american,airlines,to,blackberry10,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
14603,you,have,my,money,you,change,my,flight,and,dont,...,None,None,None,None,None,None,None,None,None,None


# Preparing data for feeding ML Algorithm (converting text into numbers)
### 5. TF-IDF
     a. Find the TF(Term Frequency)
     b. Find the IDF(Inverse Document Frequency)
     c. Find TF-IDF (Matrix multiplication of TF & IDF)

### a. Find the TF

#### i. Extract Unique tokens from all of the tokens present in the Dataset

In [17]:
%%time
words = set()
for documentTokens in lemmaTokens:
    for token in documentTokens:
        words.add(token)
uniqueWords = list(words)

Wall time: 37.9 ms


#### ii. assign all unique words as key in each document and initialize all to zero

In [18]:
%%time
documents = []

Wall time: 0 ns


In [20]:
for i in range(0,len(lemmaTokens)):
    documents.append(dict.fromkeys(uniqueWords,0))

MemoryError: ignored

In [20]:
# Preview zeros in document unique words
pd.DataFrame(documents[0:5])

,updates,awrd,lord,breath,roundtrip,horror,norfolk,inebriated,julie,sanfrancisco,ecuador,deicer,angel,ru,elaborate,pull,fehqne,executive,journey,youcandobetter,udub,nit,savr,for40,league,rdu,transit,20minute,checkin,group,wererunning,2b,sprawl,restless,boarded,four,vegetarian,gate4,ambassador,plt,...,glitchy,crave,sop,reschedule,nrt,ab,thaiairways,fligt,turquoise,willing,hint,note,strtd,300rt,sap,bcuz,robocalls,along,share,dnt,rcvd,nowarmclothes,iced,qualify,stone,majority,bs,cncdld,noair,43rd,fri,nexttime,worth,eternal,defend,daiber,thnkx,1b,sleekmoney,churn
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### iii. Calculate f<font size = 0.5>t,d</font>  `(raw count of word ‘t’ in the document ‘d’) `

In [21]:
%%time
count = 0
for documentTokens in lemmaTokens:
    for token in documentTokens:
        documents[count][token]+=1
    count+=1

CPU times: user 88.4 ms, sys: 25 µs, total: 88.4 ms
Wall time: 89.5 ms


In [22]:
%%time
# Preview raw counts
pd.DataFrame(documents)

CPU times: user 54.4 s, sys: 822 ms, total: 55.3 s
Wall time: 55.3 s


,updates,awrd,lord,breath,roundtrip,horror,norfolk,inebriated,julie,sanfrancisco,ecuador,deicer,angel,ru,elaborate,pull,fehqne,executive,journey,youcandobetter,udub,nit,savr,for40,league,rdu,transit,20minute,checkin,group,wererunning,2b,sprawl,restless,boarded,four,vegetarian,gate4,ambassador,plt,...,glitchy,crave,sop,reschedule,nrt,ab,thaiairways,fligt,turquoise,willing,hint,note,strtd,300rt,sap,bcuz,robocalls,along,share,dnt,rcvd,nowarmclothes,iced,qualify,stone,majority,bs,cncdld,noair,43rd,fri,nexttime,worth,eternal,defend,daiber,thnkx,1b,sleekmoney,churn
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14601,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14602,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14603,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### iv. Adjust the TF for document length  f<font size = 0.5>t,d</font> ÷ (number of words in d).  
<img align = left src="./tf.png"/>

In [23]:
def calculateTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        # excetiopn handled in dataset. No more handlding required.
        if bowCount == 0:
            print(wordDict)
            print(bow)
        try:
            tfDict[word] = count/float(bowCount)
        except ZeroDivisionError:
            print("found")
    return tfDict

#### v. Calculate TF in each document

In [ ]:
%%time
TF = []
count = 0
for document in documents:
    TF.append(calculateTF(document,lemmaTokens[count]))
    count += 1

In [ ]:
%%time
# Preview TF
pd.DataFrame(TF[0:10])

### b. Find the IDF
#### To calculate IDF we need the total number of document ‘N’ and dividing it by number of documents that contain a word, and calculating the logarithm of base 10.
Mathematically:

<img align = left src="./idf.png" width = 250/>

In [ ]:
def calculateIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

In [ ]:
%%time
IDFs = calculateIDF(documents)

In [ ]:
# preview IDFs
IDFs

### c. Find the TF-IDF (multiply TF and IDF) 
Mathamaticaly

<img align = left src="./tfidf.png" width = 250/>

In [ ]:
def calculateTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [ ]:
%%time
TFIDF = []
count = 0
for document in TF:
    TFIDF.append(calculateTFIDF(document,IDFs))
    count += 1

In [ ]:
len(TFIDF)

In [ ]:
%%time
# Convert list to DataFrame and Preview TF-IDF
TFIDF = pd.DataFrame(TFIDF)
#TFIDF["Output"]=labels
#TFIDF.to_csv("TFIDF.csv")
TFIDF

### Split The train test data

In [ ]:
%%time
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(TFIDF, labels, test_size=0.2, random_state=40)


### Training with Decesion Tree

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier
decisionTree = DecisionTreeClassifier()
decisionTree.fit(X_train,y_train)

### Training with Naive Bayes

In [ ]:
%%time
from sklearn.naive_bayes import MultinomialNB
naiveBayes = MultinomialNB()
naiveBayes.fit(X_train,y_train)

### Training with KNN

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)

### Predicting y' with each Classifier

In [ ]:
%%time
# Predecting 'y' with Decesion Tree
dtPreds = decisionTree.predict(X_test)

In [ ]:
%%time
# Predecting 'y' with Naive Bayes
nbPreds = naiveBayes.predict(X_test)

In [ ]:
%%time
# Predecting 'y' with K Nearest Neighbours
# Requires 8 min to predict all tests
knnPreds = knn.predict(X_test)

### Accuracy of Each Model
##### If we reduce the features in TFIDF  then we will get more accuracy

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
# Accuracy of Decesion Tree
print(confusion_matrix(y_test,dtPreds))
print(classification_report(y_test,dtPreds))
print(accuracy_score(y_test, dtPreds))

In [ ]:
# Accuracy of Naive Bayes
print(confusion_matrix(y_test,nbPreds))
print(classification_report(y_test,nbPreds))
print(accuracy_score(y_test, nbPreds))

In [ ]:
# Accuracy of K Nearest Neighbours
print(confusion_matrix(y_test,knnPreds))
print(classification_report(y_test,knnPreds))
print(accuracy_score(y_test, knnPreds))

### Ensemble Classifiers

#### 1. Voting

In [ ]:
%%time
from sklearn. ensemble import BaggingClassifier, AdaBoostClassifier, VotingClassifier
votingClf = VotingClassifier(estimators=[('decisionTree',decisionTree),('naiveBayes',naiveBayes),('knn',knn)],voting='hard')
votingClf.fit(X_train,y_train)

In [ ]:
%%time
# Requires 8 min to predict all test
votingPreds = votingClf.predict(X_test)

In [ ]:
# Accuracy of Voting Classifier
print(confusion_matrix(y_test,votingPreds))
print(classification_report(y_test,votingPreds))
print(accuracy_score(y_test, votingPreds))